In [24]:
# all the librairies to import
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
import copy
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [18]:
# other necessary conditions
pd.set_option('future.no_silent_downcasting', True)


In [19]:
data_train = pd.read_csv("train.csv", low_memory=False)

In [20]:

#training data
train_pha = data_train[["pha"]]
train_parameters = data_train.drop(["pha", "name"], axis=1)
train_parameters["neo"] = train_parameters["neo"].replace('N', 0)
train_parameters["neo"] = train_parameters["neo"].replace('Y', 1)
#test data
test_parameters = data_test.drop(["name"], axis=1)

In [21]:

train_pha_list = train_pha["pha"].tolist()
# Replace "Y" with 1 and "N" or NaN with 0
train_pha_list = [1 if x == "Y" else 0 for x in train_pha_list]


In [54]:
print(type(train_parameters))

<class 'pandas.core.frame.DataFrame'>


In [22]:

X_train, X_test, y_train, y_test = train_test_split(train_parameters, train_pha_list, test_size = 0.2, random_state = 42) # spliting the given training data
print(len(X_train))

592755


In [29]:
class MyLogisticRegression:
    def __init__(self, learning_rate = 0.01, num_iters=100):
        self.num_iters = num_iters
        self.learning_rate = learning_rate
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0

        # applying gradient descent
        for _ in range(self.num_iters):
            # initializing a linear model
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(linear_model)

            # now computing gradients
            dw = (1/num_samples) * np.dot(X.T, (y_pred - y))
            db = (1/num_samples) * np.sum(y_pred - y)

            # updating weights and biases
            self.weights -= self.learning_rate*dw
            self.bias -= self.learning_rate * db
        
    def predict(self, X_test):
        linear_model = np.dot(X_test, self.weights) + self.bias
        y_pred = self.sigmoid(linear_model)
        return y_pred


In [32]:
print(X_train.head())

               a         e          i          om           w         q  \
686177  2.234348  0.090438   5.647681  144.021842  324.002328  2.032277   
581360  2.539966  0.145925   4.587544  106.440405  142.416583  2.169321   
79532   3.234031  0.162263  19.477299  145.725496  228.038634  2.709268   
519282  2.222154  0.220185   1.884779  272.665434   68.395712  1.732870   
32924   2.702422  0.043603   5.778723  178.712231  162.698583  2.584588   

              ad     per_y  data_arc  condition_code       H neo diameter  \
686177  2.436420  3.339908    9888.0             0.0  15.800   0    1.904   
581360  2.910611  4.048089       1.0             9.0  17.686   0      NaN   
79532   3.758795  5.816002     947.0             3.0  16.403   0      NaN   
519282  2.711439  3.312604      25.0             7.0  20.186   0      NaN   
32924   2.820256  4.442607      13.0             6.0  17.977   0      NaN   

        albedo  rot_per      moid  
686177   0.256      NaN  1.052120  
581360     NaN

In [33]:
# Handle missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

[[ 2.23434847  0.09043849  5.64768134 ...  0.256      21.02042448
   1.05212   ]
 [ 2.53996593  0.14592534  4.58754408 ...  0.13032748 21.02042448
   1.15741   ]
 [ 3.23403125  0.16226291 19.47729945 ...  0.13032748 21.02042448
   1.74396   ]
 ...
 [ 2.71223615  0.11055825 10.57164257 ...  0.13032748 21.02042448
   1.4541    ]
 [ 3.11727766  0.10201593 12.31335611 ...  0.13032748 21.02042448
   1.78493   ]
 [ 2.75913685  0.17682136  9.524289   ...  0.13032748 21.02042448
   1.27071   ]]


In [34]:

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)
print(X_train_scaled)

[[-3.46358115e-03 -6.94758490e-01 -4.93827988e-01 ...  2.80251232e+00
  -3.19825172e-16 -1.64922194e-01]
 [-1.21586457e-03 -1.04745621e-01 -6.52426501e-01 ...  0.00000000e+00
  -3.19825172e-16 -1.17931615e-01]
 [ 3.88875910e-03  6.89780546e-02  1.57510870e+00 ...  0.00000000e+00
  -3.19825172e-16  1.43843711e-01]
 ...
 [ 5.11265861e-05 -4.80817434e-01  2.42805804e-01 ...  0.00000000e+00
  -3.19825172e-16  1.44801509e-02]
 [ 3.03007461e-03 -5.71651144e-01  5.03369402e-01 ...  0.00000000e+00
  -3.19825172e-16  1.62128486e-01]
 [ 3.96065928e-04  2.23783580e-01  8.61197551e-02 ...  0.00000000e+00
  -3.19825172e-16 -6.73661998e-02]]


In [35]:

# Train the custom logistic regression model
model = MyLogisticRegression(learning_rate=0.01, num_iters=100)
model.fit(X_train_scaled, y_train)


In [44]:
print(y_train)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [47]:
def convert_to_binary(predition): # dtype = list
    for i in range(len(predition)):
        if(predition[i] < 0.5):
            predition[i] = 0
        else:
            predition[i] = 1

In [49]:

# Predict and evaluate
y_train_pred = model.predict(X_train_scaled)
y_pred_binary = convert_to_binary(y_train_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy (Custom Model): {train_accuracy:.4f}")

y_test_pred = model.predict(X_test_scaled)
y_pred_binary = convert_to_binary(y_test_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy (Custom Model): {test_accuracy:.4f}")


Training Accuracy (Custom Model): 0.9976
Test Accuracy (Custom Model): 0.9975


In [63]:
# Running on the test data to create a csv file
data_test = pd.read_csv("test.csv", low_memory=False)
data_test["neo"] = data_test["neo"].replace('N', 0)
data_test["neo"] = data_test["neo"].replace('Y', 1)
#test data
data_test = data_test.drop(["name"], axis=1)

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train) # for some reason!!
X_test_imputed = imputer.transform(data_test)
# print(X_test_imputed)

X_test_scaled = scaler.transform(X_test_imputed)
print(X_test_scaled)


[[-2.15301133e-04  2.17004963e+00  4.87410735e-01 ...  0.00000000e+00
  -3.19825172e-16 -3.02548937e-01]
 [-2.98940088e-03 -1.47118250e-01 -3.48278905e-01 ...  0.00000000e+00
  -3.19825172e-16 -2.03592464e-01]
 [-2.03474671e-03 -7.99467500e-01 -4.20887403e-01 ...  0.00000000e+00
  -3.19825172e-16 -7.34402978e-02]
 ...
 [-2.80505868e-03  3.44691539e-01 -5.38217431e-01 ...  0.00000000e+00
  -3.19825172e-16 -2.32099219e-01]
 [ 2.75450183e-04 -1.24079474e+00 -2.23633572e-01 ...  0.00000000e+00
  -3.19825172e-16  9.39700550e-02]
 [-1.46602372e-04  4.55081689e-01  6.38007191e-01 ...  0.00000000e+00
  -3.19825172e-16 -1.15102094e-01]]


In [64]:
y_pred = model.predict(X_test_scaled)
print(y_pred)

[0.39503114 0.39086469 0.38983709 ... 0.39211963 0.38871156 0.39120439]


In [75]:
convert_to_binary(y_pred)
print(type(y_pred[0]))

<class 'numpy.float64'>


In [78]:
def binary_to_YN(predictions):
    # Convert binary values to 'Y' or 'N'
    return ['N' if pred == 0.0 else 'Y' for pred in predictions]

In [79]:
y_pred_yn = binary_to_YN(y_pred)
print(y_pred_yn)

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',

In [80]:

#creating a dataframe
predictions_df = pd.DataFrame(
    {
        "Predicted pha": y_pred_yn
    }
)

predictions_df.to_csv("predictions.csv", index=False)